<a href="https://colab.research.google.com/github/Om25091210/Apis-demo/blob/main/franchise_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn import linear_model

In [2]:
df=pd.read_excel(R"/content/dataStats.xlsx") #Please add the franchise sheet as a sole sheet here - upload under files section in colab
#then click on the menu of the file and then click on copy path (By default no change just add the file).

In [3]:
df.isnull().sum() #Get the total number of the null values.

,0
S.No,0
Ticket Id,0
Division,0
Corporate GST No,0
Student Name,0
Student State,0
Course Title,0
Course Type,0
Ledger Type,51
HSN code,19


In [4]:
#The number of books with a specific title people are viewing it more.
course_stock = df['Course Title'].value_counts()
print(course_stock)

Course Title
Essential in AutoCAD                                1856
Proficient in AutoCAD                                705
Essential in Revit Architecture                      483
Essential in Solidworks                              391
Course on Python                                     351
                                                    ... 
Course on Electrical Power System Analysis             1
Diploma in Artificial Intelligence                     1
Diploma in Fashion CAD ( Illustrator)                  1
Master Program in Business Intelligence                1
Course on SAP  S/4HANA Sourcing and  Procurement       1
Name: count, Length: 274, dtype: int64


In [5]:
#Division vise book purchasing.
book_divisions = df[df['Ledger Type'].isin(['Physical Book', 'E-Book'])]['Division'].value_counts()
print(book_divisions)

Division
CADD Centre Training Services Pvt Ltd    3247
Livewire India                            720
Dreamzone School Of Creative Studies      621
Synergy School Of Business Skills         433
Institute of Industrial Design             18
Name: count, dtype: int64


In [6]:
#same as above but including Course Fee.
book_divisions = df[df['Ledger Type'].isin(['Physical Book', 'E-Book', 'Course Fee'])]['Division'].value_counts()
print(book_divisions)

Division
CADD Centre Training Services Pvt Ltd    7184
Livewire India                           1726
Dreamzone School Of Creative Studies     1341
Synergy School Of Business Skills         998
Institute of Industrial Design             39
Name: count, dtype: int64


In [7]:
#Let's find out average purchase.
avg_books_per_student = df[df['Ledger Type'].isin(['Physical Book', 'E-Book'])].groupby('Student Name').size().mean()
print(avg_books_per_student)

1.2506825515016133


In [8]:
#Student state wise purchasing.
state_highest_buyers = df['Student State'].value_counts()
print(state_highest_buyers)

Student State
Tamil Nadu           4750
Kerala               2436
Maharashtra          2008
Karnataka             846
Uttar Pradesh         284
Delhi                 170
West Bengal           125
Gujarat               124
Uttarakhand           107
Haryana               105
Puducherry             89
Rajasthan              76
Assam                  65
Jammu and Kashmir      43
Jharkhand              35
Punjab                 31
Telangana              24
Bihar                   6
Madhya Pradesh          5
Himachal Pradesh        4
Andhra Pradesh          3
Tripura                 2
Odisha                  1
Name: count, dtype: int64


In [9]:
#Payment type.
most_used_payment_type = df['Payment Type'].value_counts()
print(most_used_payment_type)
print()#For better formatting.
#Payment Gateway.
most_used_payment_gateway = df['Payment Gateway'].value_counts()
print(most_used_payment_gateway)

Payment Type
Cash On Hand              6086
UPI                       3368
Debit Card                 997
Cash Deposit at Bank       364
Credit Card                259
Net Banking                205
Student Credit              41
Cheque Deposit at Bank      19
Name: count, dtype: int64

Payment Gateway
Offline                         6127
CcAvenue                        3162
PayU                            1603
AXIS Bank                        312
Axis Bank                         71
BillDesk (Includes EMI also)      64
Name: count, dtype: int64


In [10]:
#Let's get total payment approval rate.
approved_payments = df['Payout Approval'].str.contains('Approved', na=False).sum()
total_payments = len(df)

In [11]:
total_payments

11339

In [12]:
payment_approval_rate = (approved_payments / total_payments) * 100

In [13]:
payment_approval_rate

23.75870888085369

In [14]:
#Coupling things together (like we are coupled ;) )
answers = {
    "Course with highest requirements": course_stock,
    "Division buying more books": book_divisions,
    "Average books per student": avg_books_per_student,
    "State with highest buyers": state_highest_buyers,
    "Most used payment type": most_used_payment_type,
    "Most used payment gateway": most_used_payment_gateway,
    "Payment approval rate (%)": payment_approval_rate
}

print(answers)


{'Course with highest requirements': Course Title
Essential in AutoCAD                                1856
Proficient in AutoCAD                                705
Essential in Revit Architecture                      483
Essential in Solidworks                              391
Course on Python                                     351
                                                    ... 
Course on Electrical Power System Analysis             1
Diploma in Artificial Intelligence                     1
Diploma in Fashion CAD ( Illustrator)                  1
Master Program in Business Intelligence                1
Course on SAP  S/4HANA Sourcing and  Procurement       1
Name: count, Length: 274, dtype: int64, 'Division buying more books': Division
CADD Centre Training Services Pvt Ltd    7184
Livewire India                           1726
Dreamzone School Of Creative Studies     1341
Synergy School Of Business Skills         998
Institute of Industrial Design             39
Name: count, d

In [22]:
# For Physical Book or E-Book
# Filter data to include only rows related to book purchases (Physical Book, E-Book)
book_purchase_data = df[df['Ledger Type'].isin(['Physical Book', 'E-Book'])]

# Selecting relevant features for prediction
# We'll include 'Division', 'Student State', 'Payment Type', 'Course Title' and the target will be the 'Ledger Type'
features = ['Division', 'Student State', 'Payment Type', 'Course Title']
target = 'Ledger Type'

# Dropping rows with missing values in relevant columns
book_purchase_data_clean = book_purchase_data.dropna(subset=features + [target])

# Convert categorical data into numerical using one-hot encoding
book_purchase_data_encoded = pd.get_dummies(book_purchase_data_clean[features])

# Encode the target variable: 1 for 'Physical Book', 0 for 'E-Book' - resource - https://medium.com/anolytics/all-you-need-to-know-about-encoding-techniques-b3a0af68338b
book_purchase_data_clean['Ledger Type Encoded'] = book_purchase_data_clean['Ledger Type'].apply(lambda x: 1 if x == 'Physical Book' else 0)

# Define the input (X) and output (y) for the regression model
X = book_purchase_data_encoded
y = book_purchase_data_clean['Ledger Type Encoded']

# Splitting the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Applying a Logistic Regression model since the output is binary
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

accuracy


0.9275793650793651

In [16]:
report

'              precision    recall  f1-score   support\n\n           0       0.93      0.99      0.96       823\n           1       0.92      0.66      0.77       185\n\n    accuracy                           0.93      1008\n   macro avg       0.92      0.83      0.86      1008\nweighted avg       0.93      0.93      0.92      1008\n'

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

# Filter dataset for relevant features
df_payment = df[['Payment Gateway', 'Course Title', 'Payment Type', 'Division']]
df_payment.dropna(inplace=True)

# Encode categorical variables
le = LabelEncoder()
df_payment['Payment Gateway Encoded'] = le.fit_transform(df_payment['Payment Gateway'])
df_payment = pd.get_dummies(df_payment, columns=['Course Title', 'Payment Type', 'Division'])

# Train-test split
X = df_payment.drop(columns=['Payment Gateway', 'Payment Gateway Encoded'])
y = df_payment['Payment Gateway Encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Predict payment gateway for new data
gateway_pred = clf.predict(X_test)

# Decode predictions back to original labels
gateway_pred_labels = le.inverse_transform(gateway_pred)


<ipython-input-17-a193ce6ff2cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_payment.dropna(inplace=True)
<ipython-input-17-a193ce6ff2cf>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_payment['Payment Gateway Encoded'] = le.fit_transform(df_payment['Payment Gateway'])


In [18]:
gateway_pred

array([3, 3, 3, ..., 4, 4, 3])

In [19]:
gateway_pred_labels

array(['CcAvenue', 'CcAvenue', 'CcAvenue', ..., 'Offline', 'Offline',
       'CcAvenue'], dtype=object)

# **Step 1: Affiliation Impact Prediction**

1. Affiliation Impact Prediction
- To predict whether the affiliation (NSDC vs. Self) affects student behavior (e.g., enrollment numbers or completion rates), you can follow these steps:

 -- Steps:
   - Data Preparation: Use the Course Type (Self vs. NSDC Affiliation) as the target variable and identify key features such as Student State, Course Title, Course Fee, and Payment Type to serve as input features.
   - Feature Engineering: Create features like the number of students enrolled in NSDC and Self-affiliated courses, average fees, or course completion rates.
   - Modeling: Use a classification model (e.g., Logistic Regression, Random Forest, etc.) to predict whether a course is likely to be NSDC-affiliated or Self-affiliated based on the student’s preferences and regional factors.
   - Model Evaluation: Measure the model's performance using accuracy, precision, recall, or AUC-ROC, depending on the classification model chosen.

#   Example:
   - Input Features: Student State, Course Fee, Division, Payment Type.
   - Target: Course Type (NSDC vs. Self).

In [23]:
# Selecting relevant columns for the model
affiliation_df = df[['Course Type', 'Student State', 'With GST', 'Payment Type']].copy()

# Converting categorical columns into numeric (One-Hot Encoding)
affiliation_df = pd.get_dummies(affiliation_df, columns=['Student State', 'Payment Type'])

# Label encoding for the target variable 'Course Type' (NSDC affiliated vs Self-affiliated)
affiliation_df['Course Type'] = affiliation_df['Course Type'].apply(lambda x: 1 if 'NSDC' in x else 0)


In [24]:
# Splitting the data into training and testing sets
X_affiliation = affiliation_df.drop(columns=['Course Type'])
y_affiliation = affiliation_df['Course Type']

X_train_aff, X_test_aff, y_train_aff, y_test_aff = train_test_split(X_affiliation, y_affiliation, test_size=0.3, random_state=42)


In [27]:

# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Building the RandomForest model for Affiliation Impact Prediction
affiliation_model = RandomForestClassifier(random_state=42)
affiliation_model.fit(X_train_aff, y_train_aff)


RandomForestClassifier(random_state=42)

In [28]:
# Predicting on the test data
y_pred_aff = affiliation_model.predict(X_test_aff)

# Evaluating the model
affiliation_accuracy = accuracy_score(y_test_aff, y_pred_aff)
affiliation_report = classification_report(y_test_aff, y_pred_aff)

# Displaying the results
print("Affiliation Impact Prediction Accuracy:", affiliation_accuracy)
print("Classification Report:\n", affiliation_report)


Affiliation Impact Prediction Accuracy: 0.9268077601410935
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.60      0.63       352
           1       0.95      0.96      0.96      3050

    accuracy                           0.93      3402
   macro avg       0.81      0.78      0.79      3402
weighted avg       0.92      0.93      0.93      3402



# 2. Student Retention Prediction
Predicting student retention involves identifying whether a student will enroll in another course or stay within the institution for the long term.

 - Steps:
    - Data Preparation: Identify students who have enrolled in multiple courses (Student Name, Course Title, Student State can be used here).
    - Labeling: Define retention as students who have enrolled in more than one course or made multiple payments.
    - Retained students: Students who have registered for another course after completing one.
    - Non-retained students: Students who only registered once.

 - Feature Engineering: Create features like:
     - Number of Enrollments per Student.
     - Average Course Fee per Student.
     - Payment Type Preference (e.g., UPI, Net Banking).
     - Division or Course Type that the student is enrolled in.
     - Course Duration.
 - Modeling: Use a classification model to predict whether a student will be retained based on their previous behaviors, such as payment type, course fees, and state of residence.
 - Model Evaluation: Similar to the affiliation prediction, you can use accuracy, precision, recall, or AUC-ROC as evaluation metrics.

In [29]:
# Create a column to track the count of enrollments for each student
retention_df = df[['Student Name', 'Course Title', 'With GST', 'Payment Type', 'Student State']].copy()

# Group by Student Name to count the number of enrollments
retention_df['Enrollment Count'] = retention_df.groupby('Student Name')['Student Name'].transform('count')

# Define the target variable: Retention (1 if enrollment count > 1, else 0)
retention_df['Retention'] = retention_df['Enrollment Count'].apply(lambda x: 1 if x > 1 else 0)

# Drop unnecessary columns
retention_df = retention_df.drop(columns=['Student Name', 'Enrollment Count'])

# Convert categorical columns into numeric (One-Hot Encoding)
retention_df = pd.get_dummies(retention_df, columns=['Course Title', 'Payment Type', 'Student State'])


In [30]:
# Splitting the data into training and testing sets
X_retention = retention_df.drop(columns=['Retention'])
y_retention = retention_df['Retention']

X_train_ret, X_test_ret, y_train_ret, y_test_ret = train_test_split(X_retention, y_retention, test_size=0.3, random_state=42)


In [31]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


# Building the RandomForest model for Student Retention Prediction
retention_model = RandomForestClassifier(random_state=42)
retention_model.fit(X_train_ret, y_train_ret)


RandomForestClassifier(random_state=42)

In [32]:
# Predicting on the test data
y_pred_ret = retention_model.predict(X_test_ret)

# Evaluating the model
retention_accuracy = accuracy_score(y_test_ret, y_pred_ret)
retention_report = classification_report(y_test_ret, y_pred_ret)

# Displaying the results
print("Student Retention Prediction Accuracy:", retention_accuracy)
print("Classification Report:\n", retention_report)


Student Retention Prediction Accuracy: 0.8974132863021752
Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.28      0.35       339
           1       0.92      0.97      0.94      3063

    accuracy                           0.90      3402
   macro avg       0.70      0.62      0.65      3402
weighted avg       0.88      0.90      0.89      3402



# Summary
 - **Affiliation Impact: Predicts whether the course is NSDC-affiliated or Self-affiliated based on the student's details and payment information.**

 - **Student Retention: Predicts whether a student is likely to enroll in another course based on the number of courses they have already taken and other features.**